<a href="https://colab.research.google.com/github/288756/VisArtificial/blob/master/Miniproyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import drive

# Montar el Google Drive en el directorio del proyecto y descomprimir el fichero con los datos
drive.mount('/content/gdrive')
!unzip -n '/content/gdrive/My Drive/vision-artificial.zip' >> /dev/null  # ACTUALIZAR: ruta al fichero comprimido


Mounted at /content/gdrive


In [3]:
from sklearn.preprocessing import LabelEncoder

# Especificar las rutas al directorio con las imágenes y al fichero con las etiquetas
data_path = '/content/'
imgtrain_dir = data_path + "train/"
csvtrain_file = data_path + "train.csv"

imgtest_dir = data_path + "test/"
csvtest_file = data_path + "test.csv"
# Leer el fichero CSV con las etiquetas

dftrain = pd.read_csv(csvtrain_file, dtype = {"class": "category"})
dftest = pd.read_csv(csvtest_file, dtype = {"class": "category"})
dftrain = dftrain.sample(frac=1).reset_index(drop=True)  # barajar el dataframe
dftest = dftest.sample(frac=1).reset_index(drop=True)  # barajar el dataframe
# Codificar las etiquetas
label_encoder = LabelEncoder()
dftrain['class'] = label_encoder.fit_transform(dftrain['class'])
dftrain

,image_filename,mask_filename,class
0,image_015.png,mask_015.png,1
1,image_383.png,mask_383.png,1
2,image_182.png,mask_182.png,1
3,image_053.png,mask_053.png,1
4,image_054.png,mask_054.png,1
...,...,...,...
332,image_145.png,mask_145.png,1
333,image_039.png,mask_039.png,1
334,image_382.png,mask_382.png,2
335,image_062.png,mask_062.png,0


In [8]:
val_size = int(len(dftrain) * 0.2)
dftrain = dftrain.sample(frac=1).reset_index(drop=True)  # barajar el dataframe
dfval = dftrain[:val_size]
print(f'Número de ejemplos del conjunto de entrenamiento: {dftrain.shape[0]}')
print(f'Número de ejemplos del conjunto de validación: {dfval.shape[0]}')
print(f'Número de ejemplos del conjunto de test: {dftest.shape[0]}')
dftrain = dftrain.reset_index(drop=True)
dfval = dfval.reset_index(drop=True)

Número de ejemplos del conjunto de entrenamiento: 337
Número de ejemplos del conjunto de validación: 67
Número de ejemplos del conjunto de test: 113


,image_filename,mask_filename,class
0,image_242.png,mask_242.png,0
1,image_362.png,mask_362.png,0
2,image_041.png,mask_041.png,0
3,image_151.png,mask_151.png,1
4,image_351.png,mask_351.png,1
...,...,...,...
332,image_295.png,mask_295.png,0
333,image_395.png,mask_395.png,0
334,image_239.png,mask_239.png,1
335,image_416.png,mask_416.png,2


In [14]:
import tensorflow as tf
from tensorflow.data import Dataset

# Dimensiones deseadas de la imagen
img_width, img_height = 224, 224
n_channels = 3                # número de canales (RGB)
n_classes = 5                 # número de clases
x_col = 'image_filename'      # nombres de las columnas en el fichero CSV
y_col = 'class'

# Cargar y preprocesar imágenes
def load_and_preprocess_image(image_filename, label_str):
    image_path = tf.strings.join([imgtrain_dir, image_filename])
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=n_channels)
    # Redimensionar la imagen al tamaño deseado con relleno de ceros si es necesario
    image = tf.image.resize_with_pad(image, img_width, img_height)
    image = image / 255.0                               # normalización
    label = label_str                                     # No es necesario convertir a número si las etiquetas son strings
    return image, label

# Crear conjunto de datos
def get_dataset(df):
    image_filenames = df[x_col].values
    labels = df[y_col].values
    dataset = tf.data.Dataset.from_tensor_slices((image_filenames, labels))
    dataset = dataset.map(load_and_preprocess_image)
    return dataset

# Crear los conjuntos de datos y preparar los lotes
batch_size = 32
train_dataset = get_dataset(dftrain).batch(batch_size)
val_dataset = get_dataset(dfval).batch(batch_size)

print(f'Número de lotes del conjunto de entrenamiento: {len(train_dataset)}')
print(f'Número de lotes del conjunto de validación: {len(val_dataset)}')

Número de lotes del conjunto de entrenamiento: 11
Número de lotes del conjunto de validación: 3


In [15]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

def get_model():

  # Cargar la base convolucional del modelo VGG16 pre-entrenado en ImageNet
  base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width,img_height,3))

  # Ajustar los parámetros de las nuevas capas del modelo, dejando fijos los parámetros del resto de capas
  for layer in base_model.layers:
      layer.trainable = False   # por defecto, layer.trainable es True

  # Añadir nuevas capas a continuación de la base convolucional, para resolver la tarea de aprendizaje
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  x = Dense(256, activation='relu')(x)

  # Añadir una última capa completamente conectada con 5 neuronas (número de clases) para obtener la salida de la red
  predictions = Dense(n_classes, activation='softmax')(x)

  # Crear el modelo final e imprimir su representacion en modo texto
  model = Model(inputs=[base_model.input], outputs=[predictions])
  model.summary()

  return model

In [16]:
from tensorflow.keras.optimizers import Adam

# Crear el modelo
model = get_model()

# Configurar el proceso de aprendizaje
opt = Adam(learning_rate=0.001)                 # optimizador Adam
model.compile(loss='categorical_crossentropy',  # función de pérdida para problemas de clasificación multi-clase
              optimizer=opt,
              metrics=['accuracy'])

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [17]:
import numpy as np

# Entrenar el modelo con los datos preparados previamente
history = model.fit(train_dataset,
          epochs=6,   # número de epochs
          verbose=2,  # muestra información al finalizar cada epoch
          validation_data=val_dataset)

# Imprimir el error mínimo de entrenamiento y validación
train_trace = np.array(history.history['loss'])
print(f'\nError mínimo en entrenamiento: {min(train_trace):.6f}')

val_trace = np.array(history.history['val_loss'])
print(f'Error mínimo en validación: {min(val_trace):.6f}')

Epoch 1/6


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/compile_utils.py", line 277, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    